In [23]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, HuberRegressor
from sklearn.impute import KNNImputer
import joblib

In [24]:
#Baseline: 0.58990
#Data analysis
df_train = pd.read_csv("train.csv")
print(df_train.shape)

(26570, 26)


In [25]:
df_train.drop(columns='id', inplace=True)

#the correlation of each column
col_corr = df_train.corr() # -> 'loading' has the highest correlation with 'failure'
# print(col_corr)

C:\Users\User\AppData\Local\Temp\ipykernel_42264\1657787169.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  col_corr = df_train.corr() # -> 'loading' has the highest correlation with 'failure'


In [26]:
#take out the columns of 'measurement_x' and 'loading'
col_mea_load = [f for f in df_train.columns if f.startswith('measurement') or f=='loading']

#the columns mean the high correlation with 'measurement_17'
fill_dict = {}
fill_dict = {
    'A': ['measurement_5','measurement_6','measurement_7', 'measurement_8'],
    'B': ['measurement_4','measurement_5','measurement_7', 'measurement_9'],
    'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
    'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
    'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
}

for code in df_train.product_code.unique():
    curr_code_data = df_train[df_train.product_code == code]
    cur_column = fill_dict[code]
    #besides the corresponded columns, add the column 'measurement_17' and drop any row with nan.
    cur_train = curr_code_data[cur_column + ['measurement_17']].dropna(how='any')
    #cur_test take the data of the columns of the corresponding code without any nan and the column 'measurement_17' is nan
    cur_val = curr_code_data[(curr_code_data[cur_column].isnull().sum(axis = 1) == 0) & (curr_code_data['measurement_17'].isnull())]

    mode_HR = HuberRegressor()
    mode_HR.fit(cur_train[cur_column], cur_train['measurement_17'])

    df_train.loc[(df_train.product_code == code) &\
        #there is no nan in the corresponded column(cur_column)
        (df_train[cur_column].isnull().sum(axis = 1) == 0) &\
            #the value in column 'measurement_17' is nan
            (df_train['measurement_17'].isnull()), 'measurement_17'] = mode_HR.predict(cur_val[cur_column])

    #find the 5 nearest neighbors, use their values to get the mean to fill is the empty of nan.
    impute_number = KNNImputer(n_neighbors=5)
    df_train.loc[df_train.product_code == code, col_mea_load] = impute_number.fit_transform(df_train.loc[df_train.product_code == code, col_mea_load])

In [27]:
#LabelEncoder:把object type的資料轉換為數值的順序
label_encode = LabelEncoder()

df_train_cpy = df_train.copy()

col_obj = df_train.select_dtypes(object).columns

for col in col_obj:
    df_train_cpy[col] = label_encode.fit_transform(df_train[col])

df_train = df_train_cpy

In [28]:
x = df_train.drop(columns='failure')
y = df_train['failure']
# print(x.shape)
# print(y.shape)

#train_size : validation_size = 0.7 : 0.3
x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.7, random_state=60)

print("x_train's shape:", x_train.shape)
print("y_train's shape:", y_train.shape)
print("x_val's shape:", x_val.shape)
print("y_val's shape:", y_val.shape)

x_train's shape: (18599, 24)
y_train's shape: (18599,)
x_val's shape: (7971, 24)
y_val's shape: (7971,)


In [29]:
model = LogisticRegression(max_iter=3000, C=0.000005, penalty='l2', solver='newton-cg')
model.fit(x_train, y_train)

LogisticRegression(C=5e-06, max_iter=3000, solver='newton-cg')

In [30]:
#save the model
joblib.dump(model, 'my_model')

['my_model']